# Setting up an Indian Restaurant in London

### Introduction
This notebook covers the analysis of London borough data for eateries and restaurants using Foursquare datasets. It comes up with a suggestion for suitable locations for setting up a new restaurant that offers Indian cuisine.

In [6]:
import pandas as pd
import numpy as np
import requests as req

In [9]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_e01de9aea5e64c019436d63f2d3855a5 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='1IzbgPrslI8ROT5gU6NEKxRdn9x1PEm0I2z6I2ocLNHI',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_e01de9aea5e64c019436d63f2d3855a5.get_object(Bucket='loanriskassessmentcodemubi0706-donotdelete-pr-hv8bywewecfq3k',Key='london_borough.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,Borough,Governing Council,Ruling Party,Address,Area,Population,Latitude,Longitude
0,Barking and Dagenham [note 1],Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51.5607,0.1557
1,Barnet,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51.6252,-0.1517
2,Bexley,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51.4549,0.1505
3,Brent,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51.5588,-0.2817
4,Bromley,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51.4039,0.0198


In [11]:
df_1 = df_data_1.drop(['Governing Council','Ruling Party','Address','Area'], axis=1)
#df_1.columns = ['Borough', 'Population', 'LatLong']
df_1.head()

,Borough,Population,Latitude,Longitude
0,Barking and Dagenham [note 1],194352,51.5607,0.1557
1,Barnet,369088,51.6252,-0.1517
2,Bexley,236687,51.4549,0.1505
3,Brent,317264,51.5588,-0.2817
4,Bromley,317899,51.4039,0.0198


In [13]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
from geopy.geocoders import Nominatim

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  55.57 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.65 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.74 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.30 MB/s


In [15]:
address = 'London, United Kingdom'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [16]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, population in zip(df_1['Latitude'], df_1['Longitude'], df_1['Borough'], df_1['Population']):
    label = '{}, {}'.format(borough, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        weight='1',
        fill=True,
        fill_color= '#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_london)  
    
map_london

In [67]:
CLIENT_ID = '22KPZXIQGHJB2NRJ5P1QDWQPPQ0EQSUN1M0ZVRBW1U3K2CMC' # your Foursquare ID
CLIENT_SECRET = '2O1UILVIKZ3WX0XOUWS1VOYTHSWV12EDLCYSS4BJXIZEQHJ2' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22KPZXIQGHJB2NRJ5P1QDWQPPQ0EQSUN1M0ZVRBW1U3K2CMC
CLIENT_SECRET:2O1UILVIKZ3WX0XOUWS1VOYTHSWV12EDLCYSS4BJXIZEQHJ2


As a trial, let's get all the Indian Restaurants from *Ealing* within 2 KM radius

In [38]:
latitude_1 = df_1.loc[df_1['Borough'] == 'Brent ']['Latitude'].values
longitude_1 = df_1.loc[df_1['Borough'] == 'Brent ']['Longitude'].values
print(latitude_1[0], longitude_1[0])

51.5588 -0.2817


In [68]:
search_query = 'restaurant'
radius = 3000 # meters

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    latitude_1[0],
    longitude_1[0],
    VERSION,
    search_query,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=22KPZXIQGHJB2NRJ5P1QDWQPPQ0EQSUN1M0ZVRBW1U3K2CMC&client_secret=2O1UILVIKZ3WX0XOUWS1VOYTHSWV12EDLCYSS4BJXIZEQHJ2&ll=51.5588,-0.2817&v=20180604&query=restaurant&radius=3000&limit=100'

In [69]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import requests

results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
ind_rests = json_normalize(venues)
ind_rests.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,57010d89498ee7bf646e97c7,Holiday Inn London - Wembley,GB,Wembley,United Kingdom,Empire Way,673,"[Holiday Inn London - Wembley (Empire Way), We...","[{'lat': 51.55319127141448, 'lng': -0.28535934...",51.553191,-0.285359,HA9 8DS,Greater London,The Arch Bar & Restaurant,v-1549042284,NaN
1,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,5327676e498eca6fe2b70649,Wembley International Hotel,GB,Wembley,United Kingdom,Empire Way,187,"[Wembley International Hotel (Empire Way), Wem...","[{'lat': 51.559877, 'lng': -0.283784, 'label':...",51.559877,-0.283784,HA9 0NH,Greater London,Empire Restaurant,v-1549042284,NaN
2,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,5388d04a498e1afc16365310,Wembley Stadium,GB,Wembley,United Kingdom,NaN,355,"[Wembley Stadium, Wembley, Greater London, Uni...","[{'lat': 51.55607147573398, 'lng': -0.27902133...",51.556071,-0.279021,NaN,Greater London,Nets Seafood Restaurant,v-1549042284,NaN
3,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,5096a76be4b0e52fa485aae0,1 Empire Way,GB,London,United Kingdom,Empire House Business Center,323,"[1 Empire Way (Empire House Business Center), ...","[{'lat': 51.56169046915911, 'lng': -0.28213917...",51.561690,-0.282139,HA9 0EW,Greater London,Watkin's Folly Bar & Restaurant,v-1549042284,NaN
4,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,4c1d4a31b306c9289e5765b7,133 Wembley Pk Dr,GB,Wembley,United Kingdom,NaN,339,"[133 Wembley Pk Dr, Wembley, Greater London, H...","[{'lat': 51.56184458696269, 'lng': -0.28150941...",51.561845,-0.281509,HA9 8HQ,Greater London,Wembley Tandoori Restaurant,v-1549042284,NaN


In [70]:
ind_rests.shape

(37, 18)

In [43]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [90]:
# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']

nearby_venues =ind_rests.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Arch Bar & Restaurant,Restaurant,51.553191,-0.285359
1,Empire Restaurant,English Restaurant,51.559877,-0.283784
2,Nets Seafood Restaurant,Seafood Restaurant,51.556071,-0.279021
3,Watkin's Folly Bar & Restaurant,Restaurant,51.561690,-0.282139
4,Wembley Tandoori Restaurant,Indian Restaurant,51.561845,-0.281509


In [72]:
nearby_venues

,name,categories,lat,lng
0,The Arch Bar & Restaurant,Restaurant,51.553191,-0.285359
1,Empire Restaurant,English Restaurant,51.559877,-0.283784
2,Nets Seafood Restaurant,Seafood Restaurant,51.556071,-0.279021
3,Watkin's Folly Bar & Restaurant,Restaurant,51.561690,-0.282139
4,Wembley Tandoori Restaurant,Indian Restaurant,51.561845,-0.281509
5,East Pan Asian Restaurant,Asian Restaurant,51.537700,-0.301996
6,Ibis Restaurant,Restaurant,51.554942,-0.284868
7,IKEA Restaurant,Scandinavian Restaurant,51.553412,-0.259881
8,Young's (Chinese Restaurant),Asian Restaurant,51.553281,-0.284671
9,Shawarma Grill Cafe & Restaurant,Buffet,51.553574,-0.288192


In [73]:
indian_restaurants = nearby_venues[nearby_venues['categories']=='Indian Restaurant']
indian_restaurants

,name,categories,lat,lng
4,Wembley Tandoori Restaurant,Indian Restaurant,51.561845,-0.281509
15,Gana Cafe & Restaurant,Indian Restaurant,51.551232,-0.298896
17,Bobby Moore Spice,Indian Restaurant,51.558537,-0.278152
20,Apna Lounge Bar & Restaurant,Indian Restaurant,51.543026,-0.285301
21,Blue Ginger Bar & Restaurant,Indian Restaurant,51.587137,-0.302492
26,Shayona,Indian Restaurant,51.547920,-0.262541
29,Mumbai Junction Restaurant Ltd,Indian Restaurant,51.567327,-0.321929
31,Mo's Sri Lankan Restaurant,Indian Restaurant,51.584895,-0.280531


In [86]:
print("The percentage of Indian restaurants in this location is {0:.2f}%".format(len(indian_restaurants)/len(nearby_venues)*100))

The percentage of Indian restaurants in this location is 21.62%


In the above **trial** we have successfully identified the Indian restaurants for one location - Brent. The percentage of Indian restaurant in this location is 21.62%. If we determine the representation of Indian restaurant across London - for all the locations, then we can identify the locations which are poorly represented. That would be a good location for setting up one.

## Find all the Indian Restaurants across all the Borourghs
The above plot is just a proof of concept applied on Central London, within 1 KM radius. Below analysis is just the repetition of the same across all the Boroughs. Note that some boroughs are wider than 2 KM and hence it is quite possible that I miss out the venues that are away from the central point of the borough. I will fine-tune the radius and eliminate duplicate entris that might have got captured by multiple iterations.


Build a function that fetches venues given a borough name and its lat-long


In [87]:
latitudes[1:].head()

1    51.6252
2    51.4549
3    51.5588
4    51.4039
5    51.5290
Name: Latitude, dtype: float64

In [91]:
search_query = 'restaurant'
names = df_1['Borough']
latitudes = df_1['Latitude']
longitudes = df_1['Longitude']
radius = 3000
df_venues = pd.DataFrame(columns=['name', 'categories', 'lat', 'lng'])

for name, lat, lng in zip(names, latitudes, longitudes):
            
    # create the API request URL
    # url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        search_query,
        radius,
        LIMIT)
    
    # make the GET request
    # results = requests.get(url).json()["response"]['groups'][0]['items']
    results = requests.get(url).json()['response']['venues']
    
    venues = json_normalize(results)

    venues4 = venues.loc[:,['name', 'categories', 'location.lat', 'location.lng']]
    venues4['categories'] = venues4.apply(get_category_type, axis=1)
    venues4.columns = [col.split('.')[-1] for col in venues4.columns]
    venues4['borough'] = name
    
    df_venues = df_venues.append(venues4, ignore_index=True)
    
df_venues.head(10)



,borough,categories,lat,lng,name
0,Barking and Dagenham [note 1],Burger Joint,51.563225,0.146576,King's Takeaway Restaurant
1,Barking and Dagenham [note 1],Indian Restaurant,51.565345,0.171533,Saffron Restaurant
2,Barking and Dagenham [note 1],Café,51.571079,0.132994,Rose Cafe & Restaurant
3,Barking and Dagenham [note 1],Turkish Restaurant,51.570213,0.128202,Melih BBQ Restaurant
4,Barking and Dagenham [note 1],Restaurant,51.568761,0.181812,Yoo Restaurant
5,Barking and Dagenham [note 1],African Restaurant,51.542248,0.147811,Squires African Restaurant
6,Barking and Dagenham [note 1],None,51.563093,0.116370,K's Cuisine Nigerian Restaurant
7,Barking and Dagenham [note 1],Restaurant,51.575445,0.180456,Cosmo
8,Barking and Dagenham [note 1],Chinese Restaurant,51.579283,0.179579,The Shanghai
9,Barking and Dagenham [note 1],Indian Restaurant,51.574758,0.186470,Asia Spice


In [92]:
df_venues[df_venues['categories'] == 'Indian Restaurant'].shape

(146, 5)

Let's determine borough-wise the proportion of these 146 Indian restaurants in the entire restaurant population. The result of this step would be like this -

borough  |  percent
- - - - - - - - - - -
xxx      |    $.$$%

In [118]:
#Borough-wise count of all the restaurants
df_total = df_venues.groupby('borough').agg({'name': 'count'})
df_total

#Borough-wise count of Indian restaurants
df_ind_only = df_venues[df_venues['categories']=='Indian Restaurant'].groupby('borough').agg({'name': 'count'})
df_ind_only

# % representation of Indian restaurants in each bourough
df_percent = df_ind_only['name']/ df_total['name']*100
df_percent_sorted = df_percent.sort_values(ascending=True)

# Formatted print
print(df_percent_sorted.map('{0:0.2f}%'.format))


borough
Islington                            2.00%
Lambeth                              2.00%
Hackney                              4.00%
Tower Hamlets                        4.00%
Camden                               4.00%
Kensington and Chelsea               6.00%
Haringey                             6.00%
Ealing                               6.00%
Westminster                          6.00%
Barnet                               6.00%
Waltham Forest                       7.89%
Hammersmith and Fulham [note 4]      8.00%
Enfield                              8.33%
Richmond upon Thames                 8.70%
Bromley                              9.09%
Sutton                              10.53%
Croydon                             11.76%
City of London                      12.00%
Southwark                           14.00%
Newham                              14.71%
Bexley                              15.38%
Greenwich [note 2]                  16.67%
Wandsworth                          18.00%
Bar

Now let's plot these restaurants for a quick visual analysis, Indians in red and others in blue.

In [120]:
df_indian = df_venues[df_venues['categories'] == 'Indian Restaurant']
df_others = df_venues[df_venues['categories'] != 'Indian Restaurant']

# create map of New York using latitude and longitude values
map_compare = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, name in zip(df_others['lat'], df_others['lng'], df_others['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_compare)  

# add markers to map
for lat, lng, name in zip(df_indian['lat'], df_indian['lng'], df_indian['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc8631',
        fill_opacity=0.7,
        parse_html=False).add_to(map_compare)  
    
map_compare

The top choice for opening an Indian Restaurant in Islington and Lambeth, both having around 2.00% of all its restaurants as Indian. These locations are followed by Camden, Hackney and Tower Hamlets, with 4% representation.

Visual representation can help identify the low concentrations of red dots against the blue ones. Those are good locations.